In [3]:
import os
os.environ['SPARK_HOME'] = "/opt/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = "jupyter"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "lab"
os.environ['PYSPARK_PYTHON'] = "python"

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("DataFrameSQL").getOrCreate()

In [ ]:
%%bash
head -10 ./data/persons.csv

### Load Data into a DataFrame

In [6]:
# Load the synthetic data into a DataFrame
data_file_path = "./data/persons.csv"
df = spark.read.csv(data_file_path, header=True, inferSchema=True)

In [ ]:
# Display schema of DataFrame
df.printSchema()

# Show the initial DataFrame
print("Initial DataFrame:")
df.show(10)

### Register the DataFrame as a Temporary Table

In [8]:
# Register the DataFrame as a Temporary Table
df.createOrReplaceTempView("my_table")

### Perform SQL-like Queries

In [ ]:
# Select all rows where age is greater than 25
result = spark.sql("SELECT * FROM my_table WHERE age > 25")

result.show()

In [ ]:
# Compute the average salary by gender
avg_salary_by_gender = spark.sql("SELECT gender, AVG(salary) as avg_salary FROM my_table GROUP BY gender")
avg_salary_by_gender.show()

### Creating and managing temporary views.

In [11]:
# Create a temporary view
df.createOrReplaceTempView("people")

In [ ]:
# Query the temporary view
result = spark.sql("SELECT * FROM people WHERE age > 25")

result.show()

In [13]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

True

In [14]:
# Drop a temporary view
spark.catalog.dropTempView("people")

True

In [15]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

False

### Subquries

In [ ]:
# Create DataFrames
employee_data = [
    (1, "John"), (2, "Alice"), (3, "Bob"), (4, "Emily"),
    (5, "David"), (6, "Sarah"), (7, "Michael"), (8, "Lisa"),
    (9, "William")
]
employees = spark.createDataFrame(employee_data, ["id", "name"])

salary_data = [
    ("HR", 1, 60000), ("HR", 2, 55000), ("HR", 3, 58000),
    ("IT", 4, 70000), ("IT", 5, 72000), ("IT", 6, 68000),
    ("Sales", 7, 75000), ("Sales", 8, 78000), ("Sales", 9, 77000)
]
salaries = spark.createDataFrame(salary_data, ["department", "id", "salary"])

employees.show()

salaries.show()

In [17]:
# Register as temporary views
employees.createOrReplaceTempView("employees")
salaries.createOrReplaceTempView("salaries")

In [ ]:
# Subquery to find employees with salaries above average
result = spark.sql("""
    SELECT name
    FROM employees
    WHERE id IN (
        SELECT id
        FROM salaries
        WHERE salary > (SELECT AVG(salary) FROM salaries)
    )
""")

result.show()

### Window Function

In [19]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [ ]:
employee_salary = spark.sql("""
    select  salaries.*, employees.name
    from salaries 
    left join employees on salaries.id = employees.id
""")

employee_salary.show()

In [21]:
# Create a window specification
window_spec = Window.partitionBy("department").orderBy(F.desc("salary"))

In [ ]:
# Calculate the rank of employees within each department based on salary
employee_salary.withColumn("rank", F.rank().over(window_spec)).show()

In [23]:
# Stop the SparkSession
spark.stop()